In [5]:
entities_list = [
        # {
        #     "chebi_chemical_entities": {
        #         "uri": "http://purl.obolibrary.org/obo/CHEBI",
        #         "label": "chemical entities",
        #         "description": "Chemical entities in ChEBI.",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """
        #                 PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        #                 PREFIX rh:   <http://rdf.rhea-db.org/>
        #                 SELECT DISTINCT ?uri ?label
        #                 WHERE {
        #                     # Restrict to things that are recognized as Rhea reactions
        #                     ?reaction rdfs:subClassOf rh:Reaction ;
        #                             rh:side        ?side .
        #                     # Each reaction side contains one or more participants
        #                     ?side rh:contains ?participant .
        #                     # Each participant is linked to a ChEBI entity via 'rh:compound' 
        #                     ?participant rh:compound ?compound .
        #                     # 'rh:chebi' points to the ChEBI identifier (e.g., http://purl.obolibrary.org/obo/CHEBI_15377)
        #                     ?compound rh:chebi ?uri .
        #                     ?uri rdfs:label ?label .
        #                 }"""
        #     }
        # },
        # {
        #     "uniprot_disease": {
        #         "uri": "http://purl.uniprot.org/core/Disease",
        #         "label": "Disease",
        #         "description": "The preferred names of diseases.",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        #                     PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        #                     PREFIX up: <http://purl.uniprot.org/core/>
        #                     SELECT ?uri ?label ?type WHERE {
        #                         ?uri a up:Disease ;
        #                             skos:prefLabel ?label .
        #                     }"""
        #     }
        # },
        {
            "uniprot_taxon": {
                "uri": "http://purl.uniprot.org/core/Taxon",
                "label": "species",
                "description": "taxon scientific names",
                "endpoint": "https://sparql.uniprot.org/sparql/",
                "pagination": True,
                "query": """PREFIX up: <http://purl.uniprot.org/core/>
                            SELECT ?uri ?label
                            WHERE {
                                ?uri a up:Taxon ;
                                    up:scientificName ?label .
                            }"""
            }
        },
        # {
        #     "rhea_reaction": {
        #         "uri": "http://rdf.rhea-db.org/",
        #         "label": "reactions",
        #         "description": "Reactions in RHEA.",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        #                     PREFIX rh:   <http://rdf.rhea-db.org/>
        #                     SELECT DISTINCT ?uri ?label
        #                     WHERE {
        #                         ?uri rdfs:subClassOf rh:Reaction .
        #                         ?uri rdfs:label ?label .
        #                     }"""
        #     }
        # },
        
    ]

In [6]:
from endpoint_loader import *
import pandas as pd
entities = load_entities_from_endpoints(entities_list)

df = pd.DataFrame(entities)

display(df.head())

print(f"Total entities: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"Entity types: {df['entity_type'].unique()}")

Flattened 1 entities for indexing
Flattened entities: [{'uri': 'http://purl.uniprot.org/core/Taxon', 'label': 'species', 'description': 'taxon scientific names', 'endpoint': 'https://sparql.uniprot.org/sparql/', 'pagination': True, 'query': 'PREFIX up: <http://purl.uniprot.org/core/>\n                            SELECT ?uri ?label\n                            WHERE {\n                                ?uri a up:Taxon ;\n                                    up:scientificName ?label .\n                            }'}]
Loading entities from https://sparql.uniprot.org/sparql/ for species...
Found 200000 entities for species in https://sparql.uniprot.org/sparql/
  Retrieved 200000 entities, total so far: 200000
Found 200000 entities for species in https://sparql.uniprot.org/sparql/
  Retrieved 200000 entities, total so far: 400000
Found 200000 entities for species in https://sparql.uniprot.org/sparql/
  Retrieved 200000 entities, total so far: 600000
Found 200000 entities for species in https:

,label,uri,endpoint_url,entity_type,description
0,Hysteriaceae,http://purl.uniprot.org/taxonomy/100025,https://sparql.uniprot.org/sparql/,literal,taxon scientific names
1,Thomomys,http://purl.uniprot.org/taxonomy/10011,https://sparql.uniprot.org/sparql/,literal,taxon scientific names
2,Thomomys bottae,http://purl.uniprot.org/taxonomy/10013,https://sparql.uniprot.org/sparql/,literal,taxon scientific names
3,Heteromyidae,http://purl.uniprot.org/taxonomy/10015,https://sparql.uniprot.org/sparql/,literal,taxon scientific names
4,Philenoptera,http://purl.uniprot.org/taxonomy/100163,https://sparql.uniprot.org/sparql/,literal,taxon scientific names


Total entities: 2614312
Columns: ['label', 'uri', 'endpoint_url', 'entity_type', 'description']
Entity types: ['literal']


In [3]:
import pickle

df.to_csv('entities_dataframe.csv', index=False)  
with open('entities_dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)

In [2]:
import os
import pickle
if os.path.exists('entities_dataframe.pkl'):
    with open('entities_dataframe.pkl', 'rb') as f:
        df = pickle.load(f)
        entities = df.to_dict('records')
else:
    # Load from endpoint if file doesn't exist
    from endpoint_loader import *
    entities = load_entities_from_endpoints(entities_list)
    df = pd.DataFrame(entities)
    # Save for next time
    with open('entities_data.pkl', 'wb') as f:
        pickle.dump(df, f)

In [ ]:
from entity_indexing_pipeline_v3 import EmbeddingPipeline
import ray


ray.init(ignore_reinit_error=True)
s
pipeline = EmbeddingPipeline()
pipeline.add_documents(entities, batch_size=144)

ray.shutdown()

2025-03-29 10:39:10,837	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Creating embedding workers...
Initializing Qdrant client...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

Error loading upload log: Expecting ',' delimiter: line 1 column 19775128 (char 19775127)
Collection 'biomedical_entities' exists, skipping initialization
Adding 2650724 documents in batches of 144
Chunk size: 36
Time taken to generate dense embeddings with Ray Distributed Computing: 3.075160026550293 seconds
Time taken to generate sparse embeddings with Ray Distributed Computing: 13.96476697921753 seconds
Creating points...
Uploading points...
✓ Batch 1/18408 - Total uploaded: 144
Chunk size: 36
Time taken to generate dense embeddings with Ray Distributed Computing: 3.969425916671753 seconds
Time taken to generate sparse embeddings with Ray Distributed Computing: 22.855857133865356 seconds
Creating points...
Uploading points...
✓ Batch 2/18408 - Total uploaded: 288
Chunk size: 36
Time taken to generate dense embeddings with Ray Distributed Computing: 3.8981409072875977 seconds
Time taken to generate sparse embeddings with Ray Distributed Computing: 23.256271839141846 seconds
Creating 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103114790>>
Traceback (most recent call last):
  File "/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Time taken to generate sparse embeddings with Ray Distributed Computing: 8.83504581451416 seconds
Creating points...
Uploading points...
✓ Batch 7/18408 - Total uploaded: 1008
Chunk size: 36
Time taken to generate dense embeddings with Ray Distributed Computing: 4.275254964828491 seconds


In [5]:
ray.shutdown()

In [1]:
entities_list = [
        # {
        #     "chebi_chemical_entities": {
        #         "uri": "http://purl.obolibrary.org/obo/CHEBI",
        #         "label": "chemical entities",
        #         "description": "Chemical entities in ChEBI.",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """
        #                 PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        #                 PREFIX rh:   <http://rdf.rhea-db.org/>
        #                 SELECT DISTINCT ?uri ?label
        #                 WHERE {
        #                     # Restrict to things that are recognized as Rhea reactions
        #                     ?reaction rdfs:subClassOf rh:Reaction ;
        #                             rh:side        ?side .
        #                     # Each reaction side contains one or more participants
        #                     ?side rh:contains ?participant .
        #                     # Each participant is linked to a ChEBI entity via 'rh:compound' 
        #                     ?participant rh:compound ?compound .
        #                     # 'rh:chebi' points to the ChEBI identifier (e.g., http://purl.obolibrary.org/obo/CHEBI_15377)
        #                     ?compound rh:chebi ?uri .
        #                     ?uri rdfs:label ?label .
        #                 }"""
        #     }
        # },
        # {
        #     "uniprot_disease": {
        #         "uri": "http://purl.uniprot.org/core/Disease",
        #         "label": "Disease",
        #         "description": "The preferred names of diseases.",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        #                     PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        #                     PREFIX up: <http://purl.uniprot.org/core/>
        #                     SELECT ?uri ?label ?type WHERE {
        #                         ?uri a up:Disease ;
        #                             skos:prefLabel ?label .
        #                     }"""
        #     }
        # },
        # {
        #     "uniprot_taxon": {
        #         "uri": "http://purl.uniprot.org/core/Taxon",
        #         "label": "species",
        #         "description": "taxon scientific names",
        #         "endpoint": "https://sparql.uniprot.org/sparql/",
        #         "pagination": True,
        #         "query": """PREFIX up: <http://purl.uniprot.org/core/>
        #                     SELECT ?uri ?label
        #                     WHERE {
        #                         ?uri a up:Taxon ;
        #                             up:scientificName ?label .
        #                     }"""
        #     }
        # },
        {
            "rhea_reaction": {
                "uri": "http://rdf.rhea-db.org/",
                "label": "reactions",
                "description": "Reactions in RHEA.",
                "endpoint": "https://sparql.uniprot.org/sparql/",
                "pagination": True,
                "query": """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                            PREFIX rh:   <http://rdf.rhea-db.org/>
                            SELECT DISTINCT ?uri ?label
                            WHERE {
                                ?uri rdfs:subClassOf rh:Reaction .
                                ?uri rdfs:label ?label .
                            }"""
            }
        },
        
    ]

In [2]:
from endpoint_loader import *
import pandas as pd
entities = load_entities_from_endpoints(entities_list)

df = pd.DataFrame(entities)

display(df.head())
print(f"Total entities: {len(df)}")
print(f"Columns: {df.columns.tolist()}")
print(f"Entity types: {df['entity_type'].unique()}")

Flattened 1 entities for indexing
Flattened entities: [{'uri': 'http://rdf.rhea-db.org/', 'label': 'reactions', 'description': 'Reactions in RHEA.', 'endpoint': 'https://sparql.uniprot.org/sparql/', 'pagination': True, 'query': 'PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n                            PREFIX rh:   <http://rdf.rhea-db.org/>\n                            SELECT DISTINCT ?uri ?label\n                            WHERE {\n                                ?uri rdfs:subClassOf rh:Reaction .\n                                ?uri rdfs:label ?label .\n                            }'}]
Loading entities from https://sparql.uniprot.org/sparql/ for reactions...
Found 17422 entities for reactions in https://sparql.uniprot.org/sparql/
  Retrieved 17422 entities, total so far: 17422
Found 0 entities for reactions in https://sparql.uniprot.org/sparql/
Done querying SPARQL endpoints in 0.03 minutes, retrieved 17422 entities


,label,uri,endpoint_url,entity_type,description
0,(25R)-3beta-hydroxycholest-5-en-7-one-26-al + ...,http://rdf.rhea-db.org/47380,https://sparql.uniprot.org/sparql/,literal,Reactions in RHEA.
1,4-acetoxyphenol + H2O = hydroquinone + acetate...,http://rdf.rhea-db.org/47384,https://sparql.uniprot.org/sparql/,literal,Reactions in RHEA.
2,a tetradecanoate ester + H2O = an aliphatic al...,http://rdf.rhea-db.org/47388,https://sparql.uniprot.org/sparql/,literal,Reactions in RHEA.
3,hexadecanoate ester + H2O = an aliphatic alcoh...,http://rdf.rhea-db.org/47392,https://sparql.uniprot.org/sparql/,literal,Reactions in RHEA.
4,octadecanoate ester + H2O = an aliphatic alcoh...,http://rdf.rhea-db.org/47396,https://sparql.uniprot.org/sparql/,literal,Reactions in RHEA.


Total entities: 17422
Columns: ['label', 'uri', 'endpoint_url', 'entity_type', 'description']
Entity types: ['literal']


In [3]:
from entity_indexing_pipeline_v3 import EmbeddingPipeline
import ray


ray.init(ignore_reinit_error=True)
#dense_model_name="BAAI/bge-base-en-v1.5"

pipeline = EmbeddingPipeline(collection_name="biomedical_entity_collection")
pipeline.add_documents(entities, batch_size=144)

ray.shutdown()

2025-03-31 14:03:52,458	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Creating embedding workers...
Initializing Qdrant client...
Loaded 631852 previously uploaded IDs from log
Collection 'biomedical_entity_collection' exists, skipping initialization
Skipping 576 already uploaded entities
Adding 16846 documents in batches of 144
Chunk size: 36
Time taken to generate dense embeddings with Ray Distributed Computing: 10.555047273635864 seconds
Time taken to generate sparse embeddings with Ray Distributed Computing: 27.570149898529053 seconds
Creating points...
aa003d43-7f85-5d26-8ef8-ff61bfb7a5b0
ee271bf2-ec5c-5a72-b230-3f367b9f8663
2a5255b0-a105-5fe0-8b71-1f7f8a1b0595
c6c8b5a4-a79f-5175-96ca-320a744960f3
ed65eba4-ef18-5a02-a5ce-deec36fd1101
c102137e-f277-5a43-937a-e2a32aae04da
dd9807cb-3b3b-568f-b72d-a3c3087e28e7
fa197c6a-7dd8-53bc-8c69-7d84a56db4a7
57fb262c-f3a0-5bd5-8697-89fdcc04ad6c
0d5fe569-2e64-5bea-8c0a-183a20dfa1aa
15d8a708-7bdd-5377-a5d9-31f34bd7b517
df89e6a4-e6b8-5ca9-be77-8a7974436b13
49451d49-a675-5fc1-bc85-6aae0ef5d682
7a886c75-1992-5ea8-9220-9

KeyboardInterrupt: 

In [4]:
import ray

ray.shutdown()